In [ ]:
from grid.eval import *

In [ ]:
boardsize = 5
n_workers = 9

snaps = snapshots(boardsize)
snaps = pd.concat([snaps, parameters(snaps)], 1)
snaps['nickname'] = snaps.run.str.extract('.* (.*)', expand=False) + '.' + snaps.idx.astype(str)
snaps['params'] = params(snaps)
snaps = snaps.set_index('nickname')

games, wins = load(boardsize, snaps.index)

solver, soln, σ = None, None, None
futures = {}
with ProcessPoolExecutor(n_workers, initializer=init) as pool:
    while True:
        if solver is None:
            log.info('Submitting solve task')
            solver = pool.submit(activelo.solve, games, wins, soln=soln)
        elif solver.done():
            try:
                soln = solver.result()
                μ, σ = arena.analysis.difference(soln, soln.μ.idxmin())
                    
                log.info(f'coverage: {(soln.μ != 0).mean():.0%}')
                log.info(f'μ_max: {μ.max():.1f}')
                log.info(f'σ_ms: {σ.pow(2).mean()**.5:.2f}')
            except InManifoldError:
                soln = None
                log.warning('Got a manifold error; throwing soln out')
            finally:
                solver = None

        for key, future in list(futures.items()):
            if future.done():
                results = future.result()
                games, wins = update(games, wins, results)
                del futures[key]
                save(boardsize, games, wins)
                
                log.info(f'saturation: {games.sum().sum()/N_ENVS/games.shape[0]:.0%}')

        while len(futures) < n_workers - 1:
            if soln is None:
                sugg = tuple(np.random.choice(games.index, (2,)))
            else:
                sugg = suggest(soln)
            
            log.info('Submitting eval task')
            futures[sugg] = pool.submit(evaluate, *sugg)
        
snaps['μ'], snaps['σ'] = arena.analysis.difference(soln, soln.μ.idxmin())

In [ ]:
%timeit -n1 -r1 activelo.solve(games, wins, soln=soln)

In [ ]:
snaps['μ'], snaps['σ'] = arena.analysis.difference(soln, soln.μ.idxmax())

In [ ]:
plot(snaps)

In [ ]:
snaps_5 = snaps

In [ ]:
imp = activelo.improvement(soln)

In [ ]:
plt.imshow(imp/imp.max())
plt.colorbar()

In [ ]:
μ.plot()